In [25]:
import sys

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

sys.path.append('../src')
import const
from metrics import row_wise_micro_averaged_f1_score

In [26]:
train_df = pd.read_csv('../data/input/train_soundscape_labels.csv')
oof = np.load("../logs/exp_002_20210408160320_0.700/oof.npy")

In [27]:
events = oof >= 0.7
nocall_col = np.zeros((len(oof), 1)).astype(bool)
nocall_col[events.sum(1) == 0] = True
events = np.concatenate([events, nocall_col], axis=1)
oof_labels = list(
    map(
        lambda x: const.INV_BIRD_CODE[x],
        np.argwhere(events)[:, 1].reshape(-1).tolist(),
    )
)

In [28]:
train_df["oof_birds"] = oof_labels

In [14]:
train_df["nocall_birds"] = 0
train_df.loc[train_df[train_df["birds"] == "nocall"].index, "nocall_birds"] = 1

train_df["nocall_oof_birds"] = 0
train_df.loc[train_df[train_df["oof_birds"] == "nocall"].index, "nocall_oof_birds"] = 1

In [31]:
train_df["acc"] = train_df["birds"] == train_df["oof_birds"]
train_df.head()

,row_id,site,audio_id,seconds,birds,oof_birds,acc
0,7019_COR_5,COR,7019,5,nocall,nocall,True
1,7019_COR_10,COR,7019,10,nocall,nocall,True
2,7019_COR_15,COR,7019,15,nocall,nocall,True
3,7019_COR_20,COR,7019,20,nocall,nocall,True
4,7019_COR_25,COR,7019,25,nocall,nocall,True


In [40]:
train_df["diff"] = (train_df["birds"].apply(lambda x: 1 if x == "nocall" else 0) - 
                                 train_df["oof_birds"].apply(lambda x: 1 if x == "nocall" else 0))

In [41]:
train_df["diff"].value_counts()

 0    1719
-1     679
 1       2
Name: diff, dtype: int64

つまり、何かしらの鳥の鳴き声が聞こえているのに `nocall`と予測しているパターンが多いのか

In [42]:
train_df[~train_df["acc"]]

,row_id,site,audio_id,seconds,birds,oof_birds,acc,diff
240,11254_COR_5,COR,11254,5,rubwre1,nocall,False,-1
242,11254_COR_15,COR,11254,15,rubwre1,nocall,False,-1
267,11254_COR_140,COR,11254,140,obnthr1,nocall,False,-1
268,11254_COR_145,COR,11254,145,obnthr1,nocall,False,-1
269,11254_COR_150,COR,11254,150,obnthr1,nocall,False,-1
...,...,...,...,...,...,...,...,...
2390,54955_SSW_555,SSW,54955,555,chswar,nocall,False,-1
2391,54955_SSW_560,SSW,54955,560,grycat,nocall,False,-1
2394,54955_SSW_575,SSW,54955,575,chswar,nocall,False,-1
2396,54955_SSW_585,SSW,54955,585,grycat,nocall,False,-1


In [49]:
for audio_id in train_df["audio_id"].unique():
    print(f"audio_id: {audio_id}")
    print("-" * 50)
    print(train_df[train_df["audio_id"] == audio_id]['birds'].value_counts())
    print("\n")
    print(train_df[(~train_df["acc"]) & (train_df["audio_id"] == audio_id)]["diff"].value_counts())
    print("\n" * 2)

audio_id: 7019
--------------------------------------------------
nocall    120
Name: birds, dtype: int64


Series([], Name: diff, dtype: int64)



audio_id: 7954
--------------------------------------------------
nocall    120
Name: birds, dtype: int64


Series([], Name: diff, dtype: int64)



audio_id: 11254
--------------------------------------------------
nocall            105
obnthr1             6
rubwre1             5
brnjay              3
brnjay sthwoo1      1
Name: birds, dtype: int64


-1    12
Name: diff, dtype: int64



audio_id: 18003
--------------------------------------------------
rucwar                    84
nocall                    24
rucwar runwre1             3
hofwoo1                    3
hofwoo1 rucwar             2
rtlhum rucwar              1
hofwoo1 rucwar runwre1     1
grekis rucwar              1
runwre1                    1
Name: birds, dtype: int64


-1    72
 0     6
Name: diff, dtype: int64



audio_id: 21767
--------------------------------------------

In [37]:
def f(row_idx):
    preds = oof[row_idx, :]
    
    class_idx = np.argmax(preds)
    class_preds = np.max(preds)
    
    return const.INV_BIRD_CODE[class_idx], class_preds

In [39]:
f(294)

('brnjay', 0.5711570978164673)